In [1]:
n_samples = 200

In [2]:
import pandas as pd
from tqdm import tqdm
import json
from collections import defaultdict, Counter

In [3]:
train = pd.read_csv('1_my_train.csv')
test = pd.read_csv('1_my_test.csv')

In [4]:
dir = 'C:/Users/Николай/PycharmProjects/OZON E-CUP/competition/'

attributes = pd.read_parquet(f'{dir}attributes.parquet', engine='pyarrow')
attributes.set_index('variantid', inplace=True)

text = pd.read_parquet(f'{dir}text_and_bert.parquet', engine='pyarrow')
text.set_index('variantid', inplace=True)

In [5]:
category_attribute_counts = defaultdict(Counter)

In [6]:
for index, row in tqdm(test.iterrows(), total=len(test)):
    v1 = row['variantid1']
    v2 = row['variantid2']
    
    attrs1 = json.loads(attributes.loc[v1 , 'characteristic_attributes_mapping'])
    attrs2 = json.loads(attributes.loc[v2 , 'characteristic_attributes_mapping'])
    
    cat1 = json.loads(attributes.loc[v1 , 'categories'])['2']
    cat2 = json.loads(attributes.loc[v2 , 'categories'])['2']
    
    common_keys = attrs1.keys() & attrs2.keys()
    
    for key in common_keys:
        if attrs1[key] != attrs2[key]:
            # Обновление счетчиков атрибутов для категории первого товара
            category_attribute_counts[cat1].update(attrs1.keys())

100%|██████████| 49620/49620 [00:05<00:00, 9775.41it/s] 


In [7]:
top_attributes_per_category = {
    category: [attr for attr, _ in attribute_counter.most_common(n_samples)]
    for category, attribute_counter in category_attribute_counts.items()
}


In [8]:
CONFIG = {'attrs300':True}

In [9]:
# Инициализация новых колонок в DataFrame
if CONFIG.get('attrs300', False):
    def attrs300(train_df, test_df, attributes_df):
        
        # Функция для вычисления значений атрибутов
        def compute_attrs300(variantid1, variantid2, n_samples=n_samples):
            attrs1 = json.loads(attributes_df.at[variantid1, 'characteristic_attributes_mapping'])
            attrs2 = json.loads(attributes_df.at[variantid2, 'characteristic_attributes_mapping'])
            
            cat1 = json.loads(attributes_df.at[variantid1, 'categories'])['2']
            cat2 = json.loads(attributes_df.at[variantid2, 'categories'])['2']
            
            # Получение 10 наиболее частых атрибутов для данной категории
            top_attributes = top_attributes_per_category.get(cat1, [])
            
            # Инициализация результатов
            result = [-2] * n_samples # Значение по умолчанию для отсутствующих атрибутов
            
            # Сравнение значений атрибутов
            for i, attr in enumerate(top_attributes):
                if attr in attrs1 or attr in attrs2:
                    if attr in attrs1 and attr in attrs2:
                        result[i] = 1 if attrs1[attr] == attrs2[attr] else 0
                    else:
                        result[i] = -1
            
            return result

        tqdm.pandas(desc="Processing train_df")
        train_df[[f'attr{i}' for i in range(1, n_samples + 1)]] = pd.DataFrame(
            train_df.progress_apply(lambda row: compute_attrs300(row['variantid1'], row['variantid2']), axis=1).tolist(),
            index=train_df.index
        )
        
        tqdm.pandas(desc="Processing test_df")
        test_df[[f'attr{i}' for i in range(1, n_samples + 1)]] = pd.DataFrame(
            test_df.progress_apply(lambda row: compute_attrs300(row['variantid1'], row['variantid2']), axis=1).tolist(),
            index=test_df.index
        )

        return train_df, test_df

    # Применение функции к данным
    train, test = attrs300(train, test, attributes)

Processing train_df: 100%|██████████| 1250160/1250160 [01:29<00:00, 13971.13it/s]
C:\Users\Николай\AppData\Local\Temp\ipykernel_14396\2159136468.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df[[f'attr{i}' for i in range(1, n_samples + 1)]] = pd.DataFrame(
C:\Users\Николай\AppData\Local\Temp\ipykernel_14396\2159136468.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df[[f'attr{i}' for i in range(1, n_samples + 1)]] = pd.DataFrame(
C:\Users\Николай\AppData\Local\Temp\ipykernel_14396\2159136468.py:30: Perfor

In [10]:
train.to_csv('2_my_train.csv', index=False)
test.to_csv('2_my_test.csv', index=False)